In [2]:
!git clone https://github.com/shruthi-44/Vaccination-Schedule-Optimizer

Cloning into 'Vaccination-Schedule-Optimizer'...


In [3]:
from datetime import datetime, timedelta

In [25]:
VACCINE_DB = {
    "Hepatitis A": {
        "age_range": (1, 100),
        "doses": 2,
        "interval_days": 180,
        "travel_related": True,
        "contraindications": ["liver disease"],
    },
    "Yellow Fever": {
        "age_range": (9, 60),
        "doses": 1,
        "interval_days": None,
        "travel_related": True,
        "contraindications": ["immunocompromised"],
    },
    "Tdap": {
        "age_range": (10, 64),
        "doses": 1,
        "interval_days": None,
        "travel_related": False,
        "contraindications": [],
    },
    "MMR": {
        "age_range": (1, 55),
        "doses": 2,
        "interval_days": 28,
        "travel_related": False,
        "contraindications": ["pregnancy"],
    },
}


In [26]:
def check_eligibility(profile):
    eligible = []
    for vac, data in VACCINE_DB.items():
        age = profile["age"]
        if not (data["age_range"][0] <= age <= data["age_range"][1]):
            continue
        if any(cond in profile["conditions"] for cond in data["contraindications"]):
            continue
        if vac in profile["history"]:
            continue
        if data["travel_related"] and not profile.get("travel"):
            continue
        eligible.append(vac)
    return eligible


In [27]:
def priority_score(vaccine, profile, outbreak_data):
    base_score = 0
    data = VACCINE_DB[vaccine]


    if data["travel_related"] and profile.get("travel"):
        base_score += 50

    if "immunocompromised" in profile["conditions"]:
        base_score += 20


    disease = vaccine.lower()
    outbreak_risk = outbreak_data.get(disease, 0)  # e.g., from 0 to 100
    base_score += outbreak_risk

    return base_score

In [20]:
def optimize_schedule(eligible_vaccines, profile, outbreak_data):
    today = datetime.today()
    schedule = []


    scored = [(vac, priority_score(vac, profile, outbreak_data)) for vac in eligible_vaccines]
    scored.sort(key=lambda x: x[1], reverse=True)


    for vac, score in scored:
        data = VACCINE_DB[vac]
        dose_date = today
        for dose_num in range(data["doses"]):
            schedule.append({
                "vaccine": vac,
                "dose": dose_num + 1,
                "date": dose_date.strftime("%Y-%m-%d"),
                "priority_score": score
            })
            if data["interval_days"]:
                dose_date += timedelta(days=data["interval_days"])

    return schedule

In [28]:
def generate_explanations(schedule, profile, outbreak_data):
    explanations = []
    for item in schedule:
        vac = item["vaccine"]
        data = VACCINE_DB[vac]
        expl = f"{vac} dose {item['dose']} scheduled on {item['date']}."

        reasons = []
        if data["travel_related"] and profile.get("travel"):
            reasons.append(f"Travel to {', '.join(profile['travel'])} increases risk.")
        if item["priority_score"] > 50:
            reasons.append(f"High priority due to outbreak risk or vulnerability.")
        if reasons:
            expl += " Reason(s): " + "; ".join(reasons)

        explanations.append(expl)
    return explanations

In [29]:
def generate_reminders(schedule):
    reminders = []
    for item in schedule:
        reminders.append(f"Reminder: {item['vaccine']} dose {item['dose']} on {item['date']}")
    return reminders

In [30]:
def main():
    profile = {
        "name": "Sarah",
        "age": 28,
        "conditions": ["asthma"],
        "travel": ["Brazil"],
        "history": [],
    }


    outbreak_data = {
        "hepatitis a": 40,
        "yellow fever": 70,
        "tdap": 10,
        "mmr": 5,
    }

    eligible = check_eligibility(profile)
    schedule = optimize_schedule(eligible, profile, outbreak_data)
    explanations = generate_explanations(schedule, profile, outbreak_data)
    reminders = generate_reminders(schedule)

    print(f"Vaccination Schedule for {profile['name']}:\n")
    for item, expl in zip(schedule, explanations):
        print(f"- {item['vaccine']} dose {item['dose']} on {item['date']} (Priority: {item['priority_score']})")
        print(f"  Explanation: {expl}")

    print("\nReminders:")
    for r in reminders:
        print("-", r)

if __name__ == "__main__":
    main()

Vaccination Schedule for Sarah:

- Yellow Fever dose 1 on 2025-07-08 (Priority: 120)
  Explanation: Yellow Fever dose 1 scheduled on 2025-07-08. Reason(s): Travel to Brazil increases risk.; High priority due to outbreak risk or vulnerability.
- Hepatitis A dose 1 on 2025-07-08 (Priority: 90)
  Explanation: Hepatitis A dose 1 scheduled on 2025-07-08. Reason(s): Travel to Brazil increases risk.; High priority due to outbreak risk or vulnerability.
- Hepatitis A dose 2 on 2026-01-04 (Priority: 90)
  Explanation: Hepatitis A dose 2 scheduled on 2026-01-04. Reason(s): Travel to Brazil increases risk.; High priority due to outbreak risk or vulnerability.
- Tdap dose 1 on 2025-07-08 (Priority: 10)
  Explanation: Tdap dose 1 scheduled on 2025-07-08.
- MMR dose 1 on 2025-07-08 (Priority: 5)
  Explanation: MMR dose 1 scheduled on 2025-07-08.
- MMR dose 2 on 2025-08-05 (Priority: 5)
  Explanation: MMR dose 2 scheduled on 2025-08-05.

Reminders:
- Reminder: Yellow Fever dose 1 on 2025-07-08
- Remi